<a href="https://colab.research.google.com/github/ucfilho/ANN/blob/master/F20_LSHADEEpsin_F17_to_F30_cec2014.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
############################### DADOS DE ENTRADA 
function_select = 'F20'  
# this code only run from F22 to F25 (you just need to change this line the pararameters and type =cool= 
#          to go to line to change IFrank, de best or your combination
RUNS=50 #NÚMERO DE RODADAS
crossp=0.5
mut  = 0.5
freq = 1
maxNP = 240
minNP = 10
maxFES = 300000


In [2]:
problem_size = 30 # possibles 2, 10, 20, 30, 50, 100

In [3]:
############################### Importar códigos
!git clone https://github.com/ucfilho/optim_raianars_Insight/ #clonar do Github
%cd optim_raianars_Insight
import de_soma_Insight
import de_bounds_matriz
import Go2Ann
import Rand_DE
import de_bounds_best
import de_jadeequation
import lshadeepsin
import de_ann


Cloning into 'optim_raianars_Insight'...
remote: Enumerating objects: 562, done.
remote: Counting objects: 100% (248/248), done.
remote: Compressing objects: 100% (122/122), done.
remote: Total 562 (delta 156), reused 205 (delta 126), pack-reused 314
Receiving objects: 100% (562/562), 374.46 KiB | 1.94 MiB/s, done.
Resolving deltas: 100% (337/337), done.
/content/optim_raianars_Insight


In [4]:
import numpy as np
import time
import matplotlib.pyplot as plt
from random import randint
from scipy import optimize
import pandas as pd
from pandas import read_csv

In [5]:
def load_shift_data__(data_file=None):
    data = read_csv(data_file, delimiter='\s+', index_col=False, header=None)
    return data.values.reshape((-1))

def load_matrix_data__(data_file=None):
    data = read_csv(data_file, delimiter='\s+', index_col=False, header=None)
    return data.values

In [6]:
!git clone https://github.com/ucfilho/functions_cec2014_data
%cd functions_cec2014_data
import F1_to_F27_cec2014_zero_to_hero
import cec2014_names
import cec2014_shift_rotate

Cloning into 'functions_cec2014_data'...
remote: Enumerating objects: 300, done.
remote: Counting objects: 100% (6/6), done.
remote: Compressing objects: 100% (6/6), done.
remote: Total 300 (delta 2), reused 0 (delta 0), pack-reused 294
Receiving objects: 100% (300/300), 465.31 KiB | 1.88 MiB/s, done.
Resolving deltas: 100% (181/181), done.
/content/optim_raianars_Insight/functions_cec2014_data


In [7]:
dim = problem_size 

In [8]:
!pip install --upgrade --no-cache-dir gdown

In [9]:
# !gdown --id 1yM-cGzp7F35bXZ-f80uoJHzGqAb52CPz
# https://drive.google.com/file/d/1yM-cGzp7F35bXZ-f80uoJHzGqAb52CPz/view?usp=sharing
!gdown --id 1yM-cGzp7F35bXZ-f80uoJHzGqAb52CPz

/usr/local/lib/python3.7/dist-packages/gdown/cli.py:131: FutureWarning: Option `--id` was deprecated in version 4.3.1 and will be removed in 5.0. You don't need to pass it anymore to use a file ID.
  category=FutureWarning,
Downloading...
From: https://drive.google.com/uc?id=1yM-cGzp7F35bXZ-f80uoJHzGqAb52CPz
To: /content/optim_raianars_Insight/functions_cec2014_data/cec2014.zip
100% 36.1M/36.1M [00:00<00:00, 274MB/s]


In [10]:
import zipfile
file_name = zipfile.ZipFile('cec2014.zip', 'r')
file_name.extractall()

In [11]:

Function_CEC2014 =function_select + "_" + cec2014_names.NamesFunctions()[function_select]+"_D_"+str(problem_size)

f_matrix = "M_"+function_select[1:]+"_D"+str(problem_size) + ".txt"
shift_data_file="shift_data_"+function_select[1:]+".txt"
shift_data = load_shift_data__(shift_data_file)[:problem_size]
matrix = load_matrix_data__(f_matrix)
f_shuffle = "shuffle_data_"+function_select[1:]+"_D"+str(problem_size) + ".txt"
shuffle = (load_shift_data__(f_shuffle)[:problem_size] - np.ones(problem_size)).astype(int)

if(function_select=='F17'):
  f9_modified_schwefel__ = F1_to_F27_cec2014_zero_to_hero.f9_modified_schwefel__
  f1_elliptic__ = F1_to_F27_cec2014_zero_to_hero.f1_elliptic__
  f8_rastrigin__ = F1_to_F27_cec2014_zero_to_hero.f8_rastrigin__ 


  def F17(solution , shift_data= shift_data , matrix=matrix,shuffle=shuffle):
      problem_size = len(solution)
      p = np.array([0.3, 0.3, 0.4])
      n1 = int(np.ceil(p[0] * problem_size))
      n2 = int(np.ceil(p[1] * problem_size))

      idx1 = shuffle[:n1]
      idx2 = shuffle[n1:(n1+n2)]
      idx3 = shuffle[(n1+n2):]
      mz = np.dot(solution - shift_data, matrix)
      return f9_modified_schwefel__(mz[idx1]) + f8_rastrigin__(mz[idx2]) + f1_elliptic__(mz[idx3]) #+ bias

  Function = F17

if(function_select=='F18'):
  f2_bent_cigar__ = F1_to_F27_cec2014_zero_to_hero.f2_bent_cigar__
  f12_hgbat__ = F1_to_F27_cec2014_zero_to_hero.f12_hgbat__
  f8_rastrigin__ = F1_to_F27_cec2014_zero_to_hero.f8_rastrigin__ 

  def F18(solution , shift_data= shift_data , matrix=matrix,shuffle=shuffle):
      problem_size = len(solution)
      p = np.array([0.3, 0.3, 0.4])
      n1 = int(np.ceil(p[0] * problem_size))
      n2 = int(np.ceil(p[1] * problem_size))

      idx1 = shuffle[:n1]
      idx2 = shuffle[n1:(n1 + n2)]
      idx3 = shuffle[(n1 + n2):]
      mz = np.dot(solution - shift_data, matrix)
      return f2_bent_cigar__(mz[idx1]) + f12_hgbat__(mz[idx2]) + f8_rastrigin__(mz[idx3]) #+ bias

  Function = F18

if(function_select=='F19'):

  f6_weierstrass__ = F1_to_F27_cec2014_zero_to_hero.f6_weierstrass__
  f4_rosenbrock__ = F1_to_F27_cec2014_zero_to_hero.f4_rosenbrock__
  f7_griewank__ = F1_to_F27_cec2014_zero_to_hero.f7_griewank__
  f14_expanded_scaffer__ = F1_to_F27_cec2014_zero_to_hero.f14_expanded_scaffer__


  def F19(solution , shift_data= shift_data , matrix=matrix,shuffle=shuffle):
      problem_size = len(solution)
      p = np.array([0.2, 0.2, 0.3, 0.3])
      n1 = int(np.ceil(p[0] * problem_size))
      n2 = int(np.ceil(p[1] * problem_size))
      n3 = int(np.ceil(p[2] * problem_size))

      idx1 = shuffle[:n1]
      idx2 = shuffle[n1:(n1 + n2)]
      idx3 = shuffle[(n1 + n2):(n1+n2+n3)]
      idx4 = shuffle[n1+n2+n3:]
      mz = np.dot(solution - shift_data, matrix)
      return f7_griewank__(mz[idx1]) + f6_weierstrass__(mz[idx2]) + f4_rosenbrock__(mz[idx3]) + f14_expanded_scaffer__(mz[idx4]) 

  Function = F19

if(function_select=='F20'):
  f12_hgbat__ = F1_to_F27_cec2014_zero_to_hero.f12_hgbat__
  f3_discus__ = F1_to_F27_cec2014_zero_to_hero.f3_discus__
  f13_expanded_griewank__ = F1_to_F27_cec2014_zero_to_hero.f13_expanded_griewank__
  f8_rastrigin__ = F1_to_F27_cec2014_zero_to_hero.f8_rastrigin__

  def F20(solution , shift_data= shift_data , matrix=matrix,shuffle=shuffle):
      problem_size = len(solution)
      p = np.array([0.2, 0.2, 0.3, 0.3])
      n1 = int(np.ceil(p[0] * problem_size))
      n2 = int(np.ceil(p[1] * problem_size))
      n3 = int(np.ceil(p[2] * problem_size))

      idx1 = shuffle[:n1]
      idx2 = shuffle[n1:(n1 + n2)]
      idx3 = shuffle[(n1 + n2):(n1 + n2 + n3)]
      idx4 = shuffle[n1 + n2 + n3:]
      mz = np.dot(solution - shift_data, matrix)
      return f12_hgbat__(mz[idx1]) + f3_discus__(mz[idx2]) + f13_expanded_griewank__(mz[idx3]) + f8_rastrigin__(mz[idx4])# + bias

  Function = F20

if(function_select=='F21'):

  f9_modified_schwefel__ = F1_to_F27_cec2014_zero_to_hero.f9_modified_schwefel__ 
  f12_hgbat__ = F1_to_F27_cec2014_zero_to_hero.f12_hgbat__ 
  f14_expanded_scaffer__ = F1_to_F27_cec2014_zero_to_hero.f14_expanded_scaffer__  
  f1_elliptic__ = F1_to_F27_cec2014_zero_to_hero.f1_elliptic__
  f4_rosenbrock__ = F1_to_F27_cec2014_zero_to_hero.f4_rosenbrock__

  def F21(solution , shift_data= shift_data , matrix=matrix,shuffle=shuffle):
    problem_size = len(solution)
    p = np.array([0.1, 0.2, 0.2, 0.2, 0.3])
    n1 = int(np.ceil(p[0] * problem_size))
    n2 = int(np.ceil(p[1] * problem_size))
    n3 = int(np.ceil(p[2] * problem_size))
    n4 = int(np.ceil(p[3] * problem_size))

    idx1 = shuffle[:n1]
    idx2 = shuffle[n1:(n1 + n2)]
    idx3 = shuffle[(n1 + n2):(n1 + n2 + n3)]
    idx4 = shuffle[(n1+n2+n3):(n1+n2+n3+n4)]
    idx5 = shuffle[n1+n2+n3+n4:]
    mz = np.dot(solution - shift_data, matrix)

    return f14_expanded_scaffer__(mz[idx1]) + f12_hgbat__(mz[idx2]) + f4_rosenbrock__(mz[idx3]) + \
           f9_modified_schwefel__(mz[idx4]) + f1_elliptic__(mz[idx5]) 


  Function = F21

if(function_select=='F22'):

  f1_elliptic__ = F1_to_F27_cec2014_zero_to_hero.f1_elliptic__
  f10_katsuura__ = F1_to_F27_cec2014_zero_to_hero.f10_katsuura__ 
  f11_happy_cat__ = F1_to_F27_cec2014_zero_to_hero.f11_happy_cat__
  f13_expanded_griewank__ = F1_to_F27_cec2014_zero_to_hero.f13_expanded_griewank__
  f9_modified_schwefel__ = F1_to_F27_cec2014_zero_to_hero.f9_modified_schwefel__
  f5_ackley__ = F1_to_F27_cec2014_zero_to_hero.f5_ackley__

  def F22(solution , shift_data= shift_data , matrix=matrix,shuffle=shuffle):
      problem_size = len(solution)
      p = np.array([0.1, 0.2, 0.2, 0.2, 0.3])
      n1 = int(np.ceil(p[0] * problem_size))
      n2 = int(np.ceil(p[1] * problem_size))
      n3 = int(np.ceil(p[2] * problem_size))
      n4 = int(np.ceil(p[3] * problem_size))

      idx1 = shuffle[:n1]
      idx2 = shuffle[n1:(n1 + n2)]
      idx3 = shuffle[(n1 + n2):(n1 + n2 + n3)]
      idx4 = shuffle[(n1 + n2 + n3):(n1 + n2 + n3 + n4)]
      idx5 = shuffle[n1 + n2 + n3 + n4:]
      mz = np.dot(solution - shift_data, matrix)
      
      return f10_katsuura__(mz[idx1]) + f11_happy_cat__(mz[idx2]) + f13_expanded_griewank__(mz[idx3]) + \
            f9_modified_schwefel__(mz[idx4]) + f5_ackley__(mz[idx5]) #+ bias

  Function = F22

if(function_select=='F23'):
  f1_elliptic__ = F1_to_F27_cec2014_zero_to_hero.f1_elliptic__
  f2_bent_cigar__ = F1_to_F27_cec2014_zero_to_hero.f2_bent_cigar__ 
  f8_rastrigin__ = F1_to_F27_cec2014_zero_to_hero.f8_rastrigin__ 
  f3_discus__ = F1_to_F27_cec2014_zero_to_hero.f3_discus__
  f4_rosenbrock__   =  F1_to_F27_cec2014_zero_to_hero.f4_rosenbrock__


  def F23(solution , shift_data= shift_data , matrix=matrix,shuffle=shuffle):
    problem_size = len(solution)
    xichma = np.array([10, 20, 30, 40, 50])
    lamda = np.array([1, 1e-6, 1e-26, 1e-6, 1e-6])
    bias = np.array([0, 100, 200, 300, 400])

      
    # 1. Rotated Rosenbrock’s Function F4’
    t1 = solution - shift_data[0]
    g1 = lamda[0] * f4_rosenbrock__(np.dot(t1, matrix[:problem_size, :])) + bias[0]
    w1 = (1.0 / np.sqrt(sum(t1 ** 2))) * np.exp(-sum(t1 ** 2) / (2 * problem_size * xichma[0] ** 2))

    # 2. High Conditioned Elliptic Function F1’
    t2 = solution - shift_data[1]
    g2 = lamda[1] * f1_elliptic__(solution) + bias[1]
    w2 = (1.0 / np.sqrt(sum(t2 ** 2))) * np.exp(-sum(t2 ** 2) / (2 * problem_size * xichma[1] ** 2))
    # 3. Rotated Bent Cigar Function F2’
    t3 = solution - shift_data[2]
    g3 = lamda[2] * f2_bent_cigar__(np.dot(matrix[2 * problem_size: 3 * problem_size, :], t3)) + bias[2]
    w3 = (1.0 / np.sqrt(sum(t3 ** 2))) * np.exp(-sum(t3 ** 2) / (2 * problem_size * xichma[2] ** 2))
    # 4. Rotated Discus Function F3’
    t4 = solution - shift_data[3]
    g4 = lamda[3] * f3_discus__(np.dot(matrix[3 * problem_size: 4 * problem_size, :], t4)) + bias[3]
    w4 = (1.0 / np.sqrt(sum(t4 ** 2))) * np.exp(-sum(t4 ** 2) / (2 * problem_size * xichma[3] ** 2))

    # 4. High Conditioned Elliptic Function F1’
    t5 = solution - shift_data[4]
    g5 = lamda[4] * f1_elliptic__(solution) + bias[4]
    w5 = (1.0 / np.sqrt(sum(t5 ** 2))) * np.exp(-sum(t5 ** 2) / (2 * problem_size * xichma[4] ** 2))

    sw = sum([w1, w2, w3, w4, w5])
    result = (w1 * g1 + w2 * g2 + w3 * g3 + w4 * g4 + w5 * g5) / sw
    return result #+ f_bias

  Function = F23


if(function_select=='F24'):
  f1_elliptic__ = F1_to_F27_cec2014_zero_to_hero.f1_elliptic__
  f2_bent_cigar__ = F1_to_F27_cec2014_zero_to_hero.f2_bent_cigar__ 
  f8_rastrigin__ = F1_to_F27_cec2014_zero_to_hero.f8_rastrigin__ 
  f3_discus__ = F1_to_F27_cec2014_zero_to_hero.f3_discus__
  f4_rosenbrock__   =  F1_to_F27_cec2014_zero_to_hero.f4_rosenbrock__
  f1_elliptic__ = F1_to_F27_cec2014_zero_to_hero.f1_elliptic__
  f9_modified_schwefel__ = F1_to_F27_cec2014_zero_to_hero.f9_modified_schwefel__ 
  f8_rastrigin__ = F1_to_F27_cec2014_zero_to_hero.f8_rastrigin__ 
  f12_hgbat__ = F1_to_F27_cec2014_zero_to_hero.f12_hgbat__

  def F24(solution , shift_data= shift_data , matrix=matrix,shuffle=shuffle):
      problem_size = len(solution)
      xichma = np.array([20, 20, 20])
      lamda = np.array([1, 1, 1])
      bias = np.array([0, 100, 200])

      # 1. Rotated Rosenbrock’s Function F4’
      t1 = solution - shift_data[0]
      g1 = lamda[0] * f9_modified_schwefel__(solution) + bias[0]
      w1 = (1.0 / np.sqrt(sum(t1 ** 2))) * np.exp(-sum(t1 ** 2) / (2 * problem_size * xichma[0] ** 2))

      # 2. Rotated Rastrigin’s Function F9’
      t2 = solution - shift_data[1]
      g2 = lamda[1] * f8_rastrigin__(np.dot(matrix[problem_size: 2 * problem_size], t2)) + bias[1]
      w2 = (1.0 / np.sqrt(sum(t2 ** 2))) * np.exp(-sum(t2 ** 2) / (2 * problem_size * xichma[1] ** 2))

      # 3. Rotated HGBat Function F14’
      t3 = solution - shift_data[2]
      g3 = lamda[2] * f12_hgbat__(np.dot(matrix[2 * problem_size: 3 * problem_size, :], t3)) + bias[2]
      w3 = (1.0 / np.sqrt(sum(t3 ** 2))) * np.exp(-sum(t3 ** 2) / (2 * problem_size * xichma[2] ** 2))

      sw = sum([w1, w2, w3])
      result = (w1 * g1 + w2 * g2 + w3 * g3) / sw
      return result #+ f_bias

  Function = F24

 
if(function_select=='F25'):

  f9_modified_schwefel__ = F1_to_F27_cec2014_zero_to_hero.f9_modified_schwefel__ 
  f8_rastrigin__ = F1_to_F27_cec2014_zero_to_hero.f8_rastrigin__ 
  f1_elliptic__ = F1_to_F27_cec2014_zero_to_hero.f1_elliptic__

  def F25(solution , shift_data= shift_data , matrix=matrix,shuffle=shuffle):
      problem_size = len(solution)
      xichma = np.array([10, 30, 50])
      lamda = np.array([0.25, 1, 1e-7])
      bias = np.array([0, 100, 200])

      # 1. Rotated Schwefel's Function F11’
      t1 = solution - shift_data[0]
      g1 = lamda[0] * f9_modified_schwefel__(np.dot(matrix[:problem_size, :problem_size], t1)) + bias[0]
      w1 = (1.0 / np.sqrt(sum(t1 ** 2))) * np.exp(-sum(t1 ** 2) / (2 * problem_size * xichma[0] ** 2))

      # 2. Rotated Rastrigin’s Function F9’
      t2 = solution - shift_data[1]
      g2 = lamda[1] * f8_rastrigin__(np.dot(matrix[problem_size: 2 * problem_size], t2)) + bias[1]
      w2 = (1.0 / np.sqrt(sum(t2 ** 2))) * np.exp(-sum(t2 ** 2) / (2 * problem_size * xichma[1] ** 2))

      # 3. Rotated High Conditioned Elliptic Function F1'
      t3 = solution - shift_data[2]
      g3 = lamda[2] * f1_elliptic__(np.dot(matrix[2 * problem_size: 3 * problem_size, :], t3)) + bias[2]
      w3 = (1.0 / np.sqrt(sum(t3 ** 2))) * np.exp(-sum(t3 ** 2) / (2 * problem_size * xichma[2] ** 2))

      sw = sum([w1, w2, w3])
      result = (w1 * g1 + w2 * g2 + w3 * g3) / sw
      return result #+ f_bias

  Function = F25   

if(function_select=='F26'):

  f9_modified_schwefel__ = F1_to_F27_cec2014_zero_to_hero.f9_modified_schwefel__ 
  f11_happy_cat__ = F1_to_F27_cec2014_zero_to_hero.f11_happy_cat__
  f1_elliptic__ = F1_to_F27_cec2014_zero_to_hero.f1_elliptic__
  f6_weierstrass__ = F1_to_F27_cec2014_zero_to_hero.f6_weierstrass__
  f7_griewank__ = F1_to_F27_cec2014_zero_to_hero.f7_griewank__

  def F26(solution , shift_data= shift_data , matrix=matrix,shuffle=shuffle):
    problem_size = len(solution)
    xichma = np.array([10, 10, 10, 10, 10])
    lamda = np.array([0.25, 1, 1e-7, 2.5, 10])
    bias = np.array([0, 100, 200, 300, 400])

    # 1. Rotated Schwefel's Function F11’
    t1 = solution - shift_data[0]
    g1 = lamda[0] * f9_modified_schwefel__(np.dot(matrix[:problem_size, :], t1)) + bias[0]
    w1 = (1.0 / np.sqrt(sum(t1 ** 2))) * np.exp(-sum(t1 ** 2) / (2 * problem_size * xichma[0] ** 2))

    # 2. Rotated HappyCat Function F13’
    t2 = solution - shift_data[1]
    g2 = lamda[1] * f11_happy_cat__(np.dot(matrix[problem_size:2 * problem_size, :], t2)) + bias[1]
    w2 = (1.0 / np.sqrt(sum(t2 ** 2))) * np.exp(-sum(t2 ** 2) / (2 * problem_size * xichma[1] ** 2))

    # 3. Rotated High Conditioned Elliptic Function F1’
    t3 = solution - shift_data[2]
    g3 = lamda[2] * f1_elliptic__(np.dot(matrix[2 * problem_size: 3 * problem_size, :], t3)) + bias[2]
    w3 = (1.0 / np.sqrt(sum(t3 ** 2))) * np.exp(-sum(t3 ** 2) / (2 * problem_size * xichma[2] ** 2))

    # 4. Rotated Weierstrass Function F6’
    t4 = solution - shift_data[3]
    g4 = lamda[3] * f6_weierstrass__(np.dot(matrix[3 * problem_size: 4 * problem_size, :], t4)) + bias[3]
    w4 = (1.0 / np.sqrt(sum(t4 ** 2))) * np.exp(-sum(t4 ** 2) / (2 * problem_size * xichma[3] ** 2))

    # 5. Rotated Griewank’s Function F7’
    t5 = solution - shift_data[4]
    g5 = lamda[4] * f7_griewank__(np.dot(matrix[4*problem_size:, :], t5)) + bias[4]
    w5 = (1.0 / np.sqrt(sum(t5 ** 2))) * np.exp(-sum(t5 ** 2) / (2 * problem_size * xichma[4] ** 2))

    sw = sum([w1, w2, w3, w4, w5])
    result = (w1 * g1 + w2 * g2 + w3 * g3 + w4 * g4 + w5 * g5) / sw
    return result #+ f_bias

  Function = F26

if(function_select=='F27'):

  f1_elliptic__ = F1_to_F27_cec2014_zero_to_hero.f1_elliptic__
  f9_modified_schwefel__ = F1_to_F27_cec2014_zero_to_hero.f9_modified_schwefel__ 
  f12_hgbat__ = F1_to_F27_cec2014_zero_to_hero.f12_hgbat__
  f8_rastrigin__ = F1_to_F27_cec2014_zero_to_hero.f8_rastrigin__
  f6_weierstrass__ = F1_to_F27_cec2014_zero_to_hero.f6_weierstrass__

  def F27(solution , shift_data= shift_data , matrix=matrix,shuffle=shuffle):
      problem_size = len(solution)
      xichma = np.array([10, 10, 10, 20, 20])
      lamda = np.array([10, 10, 2.5, 25, 1e-6])
      bias = np.array([0, 100, 200, 300, 400])

      # 1. Rotated HGBat Function F14'
      t1 = solution - shift_data[0]
      g1 = lamda[0] * f12_hgbat__(np.dot(matrix[:problem_size, :], t1)) + bias[0]
      w1 = (1.0 / np.sqrt(sum(t1 ** 2))) * np.exp(-sum(t1 ** 2) / (2 * problem_size * xichma[0] ** 2))

      # 2. Rotated Rastrigin’s Function F9’
      t2 = solution - shift_data[1]
      g2 = lamda[1] * f8_rastrigin__(np.dot(matrix[problem_size:2 * problem_size, :], t2)) + bias[1]
      w2 = (1.0 / np.sqrt(sum(t2 ** 2))) * np.exp(-sum(t2 ** 2) / (2 * problem_size * xichma[1] ** 2))

      # 3. Rotated Schwefel's Function F11’
      t3 = solution - shift_data[2]
      g3 = lamda[2] * f9_modified_schwefel__(np.dot(matrix[2 * problem_size: 3 * problem_size, :], t3)) + bias[2]
      w3 = (1.0 / np.sqrt(sum(t3 ** 2))) * np.exp(-sum(t3 ** 2) / (2 * problem_size * xichma[2] ** 2))

      # 4. Rotated Weierstrass Function F6’
      t4 = solution - shift_data[3]
      g4 = lamda[3] * f6_weierstrass__(np.dot(matrix[3 * problem_size: 4 * problem_size, :], t4)) + bias[3]
      w4 = (1.0 / np.sqrt(sum(t4 ** 2))) * np.exp(-sum(t4 ** 2) / (2 * problem_size * xichma[3] ** 2))

      # 5. Rotated High Conditioned Elliptic Function F1’
      t5 = solution - shift_data[4]
      g5 = lamda[4] * f1_elliptic__(np.dot(matrix[4 * problem_size:, :], t5)) + bias[4]
      w5 = (1.0 / np.sqrt(sum(t5 ** 2))) * np.exp(-sum(t5 ** 2) / (2 * problem_size * xichma[4] ** 2))

      sw = sum([w1, w2, w3, w4, w5])
      result = (w1 * g1 + w2 * g2 + w3 * g3 + w4 * g4 + w5 * g5) / sw
      return result #+ f_bias

  Function = F27 

if(function_select=='F28'):

  # block of functions to F28: it use F17 and F19
  f_matrix = "M_15"+"_D"+str(problem_size) + ".txt"
  matrix_F15 = load_matrix_data__(f_matrix)

  f14_expanded_scaffer__ = F1_to_F27_cec2014_zero_to_hero.f14_expanded_scaffer__ 
  f9_modified_schwefel__ = F1_to_F27_cec2014_zero_to_hero.f9_modified_schwefel__ 
  f11_happy_cat__ = F1_to_F27_cec2014_zero_to_hero.f11_happy_cat__ 
  f1_elliptic__ = F1_to_F27_cec2014_zero_to_hero.f1_elliptic__
  f13_expanded_griewank__ = F1_to_F27_cec2014_zero_to_hero.f13_expanded_griewank__

  def F15_hybrid(solution , shift_data = shift_data, matrix = matrix_F15 , shuffle = shuffle):
      problem_size = len(solution)
      z = 5 * (solution - shift_data) / 100
      z = np.dot(z, matrix) + 1
      return f13_expanded_griewank__(z) #+ bias

  def F28(solution , shift_data= shift_data , matrix=matrix,shuffle=shuffle):
      problem_size = len(solution)
      xichma = np.array([10, 20, 30, 40, 50])
      lamda = np.array([2.5, 10, 2.5, 5e-4, 1e-6])
      bias = np.array([0, 100, 200, 300, 400])

      # 1. Rotated Expanded Griewank’s plus Rosenbrock’s Function F15’
      t1 = solution - shift_data[0]
      g1 = lamda[0] * F15_hybrid(solution) + bias[0]
      w1 = (1.0 / np.sqrt(sum(t1 ** 2))) * np.exp(-sum(t1 ** 2) / (2 * problem_size * xichma[0] ** 2))

      # 2. Rotated HappyCat Function F13’
      t2 = solution - shift_data[1]
      g2 = lamda[1] * f11_happy_cat__(np.dot(matrix[problem_size:2 * problem_size, :], t2)) + bias[1]
      w2 = (1.0 / np.sqrt(sum(t2 ** 2))) * np.exp(-sum(t2 ** 2) / (2 * problem_size * xichma[1] ** 2))

      # 3. Rotated Schwefel's Function F11’
      t3 = solution - shift_data[2]
      g3 = lamda[2] * f9_modified_schwefel__(np.dot(matrix[2 * problem_size: 3 * problem_size, :], t3)) + bias[2]
      w3 = (1.0 / np.sqrt(sum(t3 ** 2))) * np.exp(-sum(t3 ** 2) / (2 * problem_size * xichma[2] ** 2))

      # 4. Rotated Expanded Scaffer’s F6 Function F16’
      t4 = solution - shift_data[3]
      g4 = lamda[3] * f14_expanded_scaffer__(np.dot(matrix[3 * problem_size: 4 * problem_size, :], t4)) + bias[3]
      w4 = (1.0 / np.sqrt(sum(t4 ** 2))) * np.exp(-sum(t4 ** 2) / (2 * problem_size * xichma[3] ** 2))

      # 5. Rotated High Conditioned Elliptic Function F1’
      t5 = solution - shift_data[4]
      g5 = lamda[4] * f1_elliptic__(np.dot(matrix[4 * problem_size:, :], t5)) + bias[4]
      w5 = (1.0 / np.sqrt(sum(t5 ** 2))) * np.exp(-sum(t5 ** 2) / (2 * problem_size * xichma[4] ** 2))

      sw = sum([w1, w2, w3, w4, w5])
      result = (w1 * g1 + w2 * g2 + w3 * g3 + w4 * g4 + w5 * g5) / sw
      return result #+ f_bias

  Function = F28 

if(function_select=='F29'):
  # block of functions to F28: it use F17 and F19
  f_matrix = "M_17"+"_D"+str(problem_size) + ".txt"
  matrix_F17 = load_matrix_data__(f_matrix)

  f_matrix = "M_18"+"_D"+str(problem_size) + ".txt"
  matrix_F18 = load_matrix_data__(f_matrix)

  f_matrix = "M_19"+"_D"+str(problem_size) + ".txt"
  matrix_F19 = load_matrix_data__(f_matrix)

  f9_modified_schwefel__  = F1_to_F27_cec2014_zero_to_hero.f9_modified_schwefel__ 
  f8_rastrigin__ = F1_to_F27_cec2014_zero_to_hero.f8_rastrigin__
  f1_elliptic__ = F1_to_F27_cec2014_zero_to_hero.f1_elliptic__

  f2_bent_cigar__ = F1_to_F27_cec2014_zero_to_hero.f2_bent_cigar__
  f12_hgbat__ = F1_to_F27_cec2014_zero_to_hero.f12_hgbat__
  #f8_rastrigin__ = F1_to_F27_cec2014_zero_to_hero.f8_rastrigin__

  f7_griewank__ = F1_to_F27_cec2014_zero_to_hero.f7_griewank__
  f6_weierstrass__ = F1_to_F27_cec2014_zero_to_hero.f6_weierstrass__
  f4_rosenbrock__ = F1_to_F27_cec2014_zero_to_hero.f4_rosenbrock__
  f14_expanded_scaffer__ = F1_to_F27_cec2014_zero_to_hero.f14_expanded_scaffer__ 

  def F17_hybrid(solution , shift_data = shift_data, matrix = matrix_F17 , shuffle = shuffle):
      problem_size = len(solution)
      p = np.array([0.3, 0.3, 0.4])
      n1 = int(np.ceil(p[0] * problem_size))
      n2 = int(np.ceil(p[1] * problem_size))
      idx1 = shuffle[:n1]
      idx2 = shuffle[n1:(n1+n2)]
      idx3 = shuffle[(n1+n2):]
      mz = np.dot(solution - shift_data, matrix)

      return f9_modified_schwefel__(mz[idx1]) + f8_rastrigin__(mz[idx2]) + f1_elliptic__(mz[idx3]) #+ bias

  def F18_hybrid(solution , shift_data = shift_data, matrix = matrix_F17 , shuffle = shuffle):
      problem_size = len(solution)
      p = np.array([0.3, 0.3, 0.4])
      n1 = int(np.ceil(p[0] * problem_size))
      n2 = int(np.ceil(p[1] * problem_size))
      idx1 = shuffle[:n1]
      idx2 = shuffle[n1:(n1 + n2)]
      idx3 = shuffle[(n1 + n2):]
      mz = np.dot(solution - shift_data, matrix)
      return f2_bent_cigar__(mz[idx1]) + f12_hgbat__(mz[idx2]) + f8_rastrigin__(mz[idx3]) #+ bias 

  def F19_hybrid(solution , shift_data = shift_data, matrix = matrix_F17 , shuffle = shuffle):
      problem_size = len(solution)
      p = np.array([0.2, 0.2, 0.3, 0.3])
      n1 = int(np.ceil(p[0] * problem_size))
      n2 = int(np.ceil(p[1] * problem_size))
      n3 = int(np.ceil(p[2] * problem_size))
      idx1 = shuffle[:n1]
      idx2 = shuffle[n1:(n1 + n2)]
      idx3 = shuffle[(n1 + n2):(n1+n2+n3)]
      idx4 = shuffle[n1+n2+n3:]
      mz = np.dot(solution - shift_data, matrix)
      return f7_griewank__(mz[idx1]) + f6_weierstrass__(mz[idx2]) + f4_rosenbrock__(mz[idx3]) + f14_expanded_scaffer__(mz[idx4])#+ bias


  def F29(solution , shift_data= shift_data , matrix=matrix,shuffle=shuffle):
      num_funcs = 3
      problem_size = len(solution)
      xichma = np.array([10, 30, 50])
      lamda = np.array([1, 1, 1])
      bias = np.array([0, 100, 200])

      def __fi__(solution=None, idx=None):
          if idx == 0:
              return F17_hybrid(solution) #F17_hybrid(solution , shift_data , matrix, shuffle)
          elif idx == 1:
              return F18_hybrid(solution )
          else:
              return F19_hybrid(solution)

      weights = np.ones(num_funcs)
      fits = np.ones(num_funcs)
      for i in range(0, num_funcs):
          t1 = lamda[i] * __fi__(solution, i) + bias[i]
          t2 = 1.0 / np.sqrt(sum((solution - shift_data[i]) ** 2))
          w_i = t2 * np.exp(-sum((solution - shift_data[i]) ** 2) / (2 * problem_size * xichma[i] ** 2))
          weights[i] = w_i
          fits[i] = t1
      sw = sum(weights)
      result = 0.0
      for i in range(0, num_funcs):
          result += (weights[i] / sw) * fits[i]
      return result #+ f_bias

  Function = F29 

if(function_select=='F30'):
  f12_hgbat__ = F1_to_F27_cec2014_zero_to_hero.f12_hgbat__ 
  f3_discus__ = F1_to_F27_cec2014_zero_to_hero.f3_discus__ 
  f13_expanded_griewank__ = F1_to_F27_cec2014_zero_to_hero.f13_expanded_griewank__ 
  f8_rastrigin__ = F1_to_F27_cec2014_zero_to_hero.f8_rastrigin__ 


  f14_expanded_scaffer__ = F1_to_F27_cec2014_zero_to_hero.f14_expanded_scaffer__ 
  #f12_hgbat__ = F1_to_F27_cec2014_zero_to_hero.f12_hgbat__
  f4_rosenbrock__ = F1_to_F27_cec2014_zero_to_hero.f4_rosenbrock__
  f9_modified_schwefel__ = F1_to_F27_cec2014_zero_to_hero.f9_modified_schwefel__
  f1_elliptic__ = F1_to_F27_cec2014_zero_to_hero.f1_elliptic__  

  f10_katsuura__ = F1_to_F27_cec2014_zero_to_hero.f10_katsuura__
  f11_happy_cat__ = F1_to_F27_cec2014_zero_to_hero.f11_happy_cat__
  #f13_expanded_griewank__ = F1_to_F27_cec2014_zero_to_hero.f13_expanded_griewank__
  #f9_modified_schwefel__ = F1_to_F27_cec2014_zero_to_hero.f9_modified_schwefel__
  f5_ackley__ = F1_to_F27_cec2014_zero_to_hero.f5_ackley__ 

  # block of functions to F30: it use F20, F21, F22
  f_matrix = "M_20"+"_D"+str(problem_size) + ".txt"
  matrix_F20 = load_matrix_data__(f_matrix)
  f_matrix = "M_21"+"_D"+str(problem_size) + ".txt"
  matrix_F21 = load_matrix_data__(f_matrix)
  f_matrix = "M_22"+"_D"+str(problem_size) + ".txt"
  matrix_F22 = load_matrix_data__(f_matrix)

  def F20_hybrid(solution , shift_data = shift_data, matrix = matrix_F20 , shuffle = shuffle):
      problem_size = len(solution)
      p = np.array([0.2, 0.2, 0.3, 0.3])
      n1 = int(np.ceil(p[0] * problem_size))
      n2 = int(np.ceil(p[1] * problem_size))
      n3 = int(np.ceil(p[2] * problem_size))

      shuffle = (shuffle[:problem_size] - np.ones(problem_size)).astype(int)
      idx1 = shuffle[:n1]
      idx2 = shuffle[n1:(n1 + n2)]
      idx3 = shuffle[(n1 + n2):(n1 + n2 + n3)]
      idx4 = shuffle[n1 + n2 + n3:]
      mz = np.dot(solution - shift_data, matrix)
      return f12_hgbat__(mz[idx1]) + f3_discus__(mz[idx2]) + f13_expanded_griewank__(mz[idx3]) + f8_rastrigin__(mz[idx4]) #+ bias


  def F21_hybrid(solution , shift_data = shift_data, matrix = matrix_F21 , shuffle = shuffle):
      problem_size = len(solution)
      p = np.array([0.1, 0.2, 0.2, 0.2, 0.3])
      n1 = int(np.ceil(p[0] * problem_size))
      n2 = int(np.ceil(p[1] * problem_size))
      n3 = int(np.ceil(p[2] * problem_size))
      n4 = int(np.ceil(p[3] * problem_size))

      shuffle = (shuffle[:problem_size] - np.ones(problem_size)).astype(int)

      idx1 = shuffle[:n1]
      idx2 = shuffle[n1:(n1 + n2)]
      idx3 = shuffle[(n1 + n2):(n1 + n2 + n3)]
      idx4 = shuffle[(n1+n2+n3):(n1+n2+n3+n4)]
      idx5 = shuffle[n1+n2+n3+n4:]
      mz = np.dot(solution - shift_data, matrix)
      return f14_expanded_scaffer__(mz[idx1]) + f12_hgbat__(mz[idx2]) + f4_rosenbrock__(mz[idx3]) + \
            f9_modified_schwefel__(mz[idx4]) + f1_elliptic__(mz[idx5]) #+ bias


  def F22_hybrid(solution , shift_data = shift_data, matrix = matrix_F22 , shuffle = shuffle):
      problem_size = len(solution)
      p = np.array([0.1, 0.2, 0.2, 0.2, 0.3])
      n1 = int(np.ceil(p[0] * problem_size))
      n2 = int(np.ceil(p[1] * problem_size))
      n3 = int(np.ceil(p[2] * problem_size))
      n4 = int(np.ceil(p[3] * problem_size))

      idx1 = shuffle[:n1]
      idx2 = shuffle[n1:(n1 + n2)]
      idx3 = shuffle[(n1 + n2):(n1 + n2 + n3)]
      idx4 = shuffle[(n1 + n2 + n3):(n1 + n2 + n3 + n4)]
      idx5 = shuffle[n1 + n2 + n3 + n4:]
      mz = np.dot(solution - shift_data, matrix)
      return f10_katsuura__(mz[idx1]) + f11_happy_cat__(mz[idx2]) + f13_expanded_griewank__(mz[idx3]) + \
            f9_modified_schwefel__(mz[idx4]) + f5_ackley__(mz[idx5]) #+ 

  def F30(solution , shift_data= shift_data , matrix=matrix,shuffle=shuffle):
      num_funcs = 3
      problem_size = len(solution)
      xichma = np.array([10, 30, 50])
      lamda = np.array([1, 1, 1])
      bias = np.array([0, 100, 200])

      def __fi__(solution=None, idx=None):
          if idx == 0:
              return F20_hybrid(solution)
          elif idx == 1:
              return F21_hybrid(solution)
          else:
              return F22_hybrid(solution)

      weights = np.ones(num_funcs)
      fits = np.ones(num_funcs)
      for i in range(0, num_funcs):
          t1 = lamda[i] * __fi__(solution, i) + bias[i]
          t2 = 1.0 / np.sqrt(sum((solution - shift_data[i]) ** 2))
          w_i = t2 * np.exp(-sum((solution - shift_data[i]) ** 2) / (2 * problem_size * xichma[i] ** 2))
          weights[i] = w_i
          fits[i] = t1
      sw = sum(weights)
      result = 0.0
      for i in range(0, num_funcs):
          result += (weights[i] / sw) * fits[i]
      return result #+ f_bias

  Function = F30 

In [12]:
def load_shift_data__(data_file=None):
    data = read_csv(data_file, delimiter='\s+', index_col=False, header=None)
    return data.values.reshape((-1))

def load_matrix_data__(data_file=None):
    data = read_csv(data_file, delimiter='\s+', index_col=False, header=None)
    return data.values

In [13]:
fobj=Function
Fun = Function

In [14]:
!git clone https://github.com/ucfilho/Raianars_paper_One #clonar do Github
%cd Raianars_paper_One

Cloning into 'Raianars_paper_One'...
remote: Enumerating objects: 277, done.
remote: Counting objects: 100% (148/148), done.
remote: Compressing objects: 100% (65/65), done.
remote: Total 277 (delta 83), reused 142 (delta 82), pack-reused 129
Receiving objects: 100% (277/277), 667.30 KiB | 2.60 MiB/s, done.
Resolving deltas: 100% (155/155), done.
/content/optim_raianars_Insight/functions_cec2014_data/Raianars_paper_One


In [15]:
MAX=np.repeat(100,dim)
MIN=np.repeat(-100,dim)


In [ ]:
start_time = time.time()
FUNCT =[]
VALUES =[]

for ii in range(RUNS):
    gen = 0
    FES = 0
    popsize = maxNP
    SF =[mut];SCR=[crossp];SFreq=[freq]
    miValues= [[mut], [crossp], [freq]] 
    MAT=[] 
    method= [maxNP,minNP,maxFES,FES,SF,SCR,SFreq,miValues] 
   
    X=np.zeros((popsize,dim)) # X
    for i in range(popsize):
      for j in range(dim):
        r=np.random.random()
        X[i,j]=r*(MAX[j]-MIN[j])+MIN[j]

    fitness = np.asarray([fobj(ind) for ind in X])
    FES = FES + popsize

    while(FES < maxFES):

      
      ############################################################################################
      ##=cool=#####################################################################################
      X,BEST,FBEST,XY,BEST_XY,tuneEVAL,SF,SCR,SFreq,miF,miCR,miFreq = lshadeepsin.de(MAX,MIN,gen,popsize,fobj,X,fitness,method)                                                                     
      popsize,FES,gen=tuneEVAL 
      if(gen % 500 == 0):
        print('run=',ii,'eval funct =',FES,'gen=',gen,'Fobj=',FBEST)
      ############################################################################################
      method = [maxNP,minNP,maxFES,FES,SF,SCR,SFreq,miValues] 
      fitness = XY[:,-1]



    print('run =',ii,'time=',(time.time()-start_time)/60, 'popsize=',popsize,'FES=',FES,'FO=',FBEST)  
    FUNCT.append(FBEST)
    VALUES.append(BEST)
    
    #print("GlobalParam e Solucao:", BEST,"Funcao=",FOBEST)
    




  

run= 0 eval funct = 54460 gen= 500 Fobj= 1.8960442810015862e+18


In [ ]:
popsize

In [ ]:
############################## Cálculo dos Parâmetros de Avaliação   
TIME = (time.time() - start_time)/RUNS
FUNCT = np.asarray(FUNCT)
best_idx = np.argmin(FUNCT) # not using the best anymore but a random best
BEST_AGENTS = np.asarray(VALUES[best_idx])
BEST = FUNCT[best_idx]
FUNCT_ST= FUNCT.std()
MEAN=np.mean(FUNCT)
MAXFO=FUNCT.max()
MINFO=FUNCT.min()

In [ ]:
nome='LShade'+'_Fo-'+str(mut)+'_CRo-'+str(crossp)+'_D-'+str(dim)+'_evaluetions-'+str(maxFES)
print('lshadeEPsin','maxNP=',maxNP,'minNP=',minNP,'Dim=', dim,"Min/Max=",MIN[0],"/",MAX[0],'_',Fun.__name__)
print(nome, '\n')
print('MÉDIA: ',MEAN,'STD __:',FUNCT_ST)
print('MIN __:',MINFO, 'MAX__: ',MAXFO)
print(MEAN)
print(FUNCT_ST)
print(MINFO)
print("Tempo Médio= %.3f segundos ---" % (TIME))
print(RUNS,'Rodadas') 
print('BEST: ',BEST_AGENTS)
